## Dataset Generation
You can do dynamic generation on the go, but it usually takes around 1.5 hours to go through the dataset. I prefer to use statically generated dataset because it allows to iterate throught the dataset in am matter of 4 minutes. Frame size in the number of actions you want to be in the state (the number of movies rated). I have chosen it to be 10, if you want another number you will have to generate it yourself. Link to the google drive is in the readme.md


## This is a mandatory thing to do, you can download the existing dataset here: [Link](https://drive.google.com/open?id=1pPf-7AmUVceVfgfmKEJ6ireEDKEJHw-7)



### Things you will need to download:
- [Movie embeddings](https://drive.google.com/open?id=1kTyu05ZmtP2MA33J5hWdX8OyUYEDW4iI) or you can generate them yourself
- [State Representation](https://drive.google.com/open?id=1DuNvPQ8pIxmZEFGNtXRSRxRcoWXU_0cO) if you want to generate the dataset LITE by encoding sequential features into lower dimensions 1290 -> 256

## (Optional) Trim Ratings
Exlude films with frequency <50

In [ ]:
import pandas as pd
ratings = pd.read_csv("../data/ml-20m/ratings.csv")
s = pd.Series(ratings['movieId'].value_counts())
# here is the value you can change
to_ignore = s.loc[s.where(lambda s: s <= 50).isna()].index.values
ratings = ratings[ratings['movieId'].isin(to_ignore)]
ratings.to_csv('../data/ml-20m/ratings_lite.csv', index=False)

## Dataset Genaration

In [ ]:
import torch
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook as tqdm
import json
import pickle

## Settings here!
SMALL_STATE = True 
frame_size = 10
batch_size = 1000
## End settings

cuda = torch.device('cpu')
ratings = pd.read_csv('../data/ml-20m/ratings.csv')
movies = pickle.load(open('../data/infos_pca128.pytorch', 'rb'))

# credits: KnightofK9
ratings["rating"] = ratings["rating"].apply(lambda i: 2 * (i - 2.5))
users = ratings[["userId","movieId"]].groupby(["userId"]).size()
users = users[users >= frame_size + 1]

In [ ]:
ratings['movieId'].max()

In [ ]:
ratings = ratings.sort_values(by=["userId", "timestamp"]).drop(columns=["timestamp"]).set_index("userId")

for i in movies.keys():
    movies[i] = movies[i].to(cuda)

In [ ]:
# if SMALL_STATE is False, there is no need to run this cell
import torch.nn as nn

class StateRepresentation(nn.Module):
    def __init__(self):
        super(StateRepresentation, self).__init__()
        self.lin = nn.Sequential(
            # 128 - embed size, 1 - rating size
            nn.Linear(frame_size * (128 + 1), 256),
            nn.Tanh(),
        )
        
    def forward(self, state):
        # apply state represemtation module
        state = self.lin(state)
        return state
    
state_rep = StateRepresentation()
state_rep.load_state_dict(torch.load('../models/state_rep.pt'))
state_rep.eval()

In [ ]:
import h5py
import numpy as np
import torch
import pickle

h5 = h5py.File("test.hdf5", "w")
idx = 0
# datasets:
# ds_state (float) (None, (frame_size + 1) * embed_size): (None, 1290)
# ds_action (float) (None, embed_size): (None, 128)
# ds_reward (int4) (None)
# ds_next_state (float) (None, (frame_size + 1) * embed_size): (None, 1290)
# ds_done (bool) (None)

In [ ]:
ds_ratings = h5.create_dataset("ratings", (0, 11), maxshape=(None, 256), dtype='int8', chunks=True, compression="lzf")
ds_movies = h5.create_dataset("movies", (0, 11), maxshape=(None, 256), dtype='int32',  chunks=True, compression="lzf")

ds_done = h5.create_dataset("done", (0, ), maxshape=(None,), dtype='?', chunks=True,  compression="lzf")

In [ ]:
import io
import matplotlib.pyplot as plt
import pickle

n_iter = 1
n_batch = 0
n_user = 0


def get_minibatch(df, idx):
    user_ratings = df[idx:frame_size + idx + 1]
    user_ratings = user_ratings[["movieId", "rating"]].values

    movies = user_ratings[:, 0]
    ratings = user_ratings[:, 1]

    # state action reward next_state done
    return [movies, ratings, idx + 1 == size]


batch_bar = tqdm(total=len(users))
batch = []

g_idx = 0

for user in ratings.index.unique():
    df = ratings.loc[user]
    batch_bar.update(1)
    n_batch += 1
    size = max(len(df) - frame_size, 0)
    for idx in range(0, size):
        # another value you can tweak!
        if np.random.rand() < 0.8:  # intake percents
            continue
        batch.append(get_minibatch(df, idx))
        
        if len(batch) % batch_size == 0:
            
            [i.resize([len(batch) + i.shape[0], i.shape[1]]) for i in [ds_ratings, ds_movies]]
            [i.resize([len(batch) + i.shape[0]]) for i in [ds_done]]
            
            for i in batch:
                movies, ratings_, done = i
                # frame_size, embed_size -> frame_size * embed_size
                # infos = infos.view(-1).numpy()

                movies = np.array(movies)
                ratings_ = np.array(ratings_)
                done = np.array(done)

                ds_movies[g_idx] = movies.astype('int32')
                ds_ratings[g_idx] = ratings_.astype('int8')
                ds_done[g_idx] = done.astype('?')
                #ds_next_state[g_idx] = next_state.astype('float16')
                #ds_done[g_idx] = done
                g_idx+=1
            
            del batch
            batch = []
            


## Test:

In [ ]:
import torch
import pickle
from tqdm import tqdm_notebook as tqdm
import h5py
import numpy as np

movie_ref = pickle.load(open('../data/infos_pca128.pytorch', 'rb'))

f = h5py.File("test.hdf5", "r")

cuda = torch.device('cuda')

def prepare_batch(*args):
    args = [torch.tensor(np.array(arg).astype(np.float)).to(cuda) for arg in args]
    return args


batch_size = 5000
losses = []
T_losses = []


for i in tqdm(range(f['done'].shape[0] // batch_size)):
    movies, ratings, done = [f[key][i*batch_size:(i+1)*batch_size] for key in
             ['movies', 'ratings', 'done']]
    
    movies, ratings, done = [torch.tensor(i.astype('float32')) for i in [movies, ratings, done]]
    movies_tensor = torch.stack([torch.stack([movie_ref[int(i)] for i in ts]) for ts in movies])
    
    state = torch.cat([movies_tensor[:, :-1, :].view(state.size(0), -1),
                       ratings[:, :-1]], 1)
    next_state = torch.cat([movies_tensor[:, 1:, :].view(state.size(0), -1),
                            ratings[:, 1:]], 1)
    action = movies_tensor[:, -1]
    reward = ratings[:, -1]
